In [1]:
import os
import sys
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
%load_ext autoreload

import torch
import sagemaker
from sagemaker.pytorch import PyTorch

module_path = os.path.abspath(os.path.join('../py-conjugated/networks/'))
if module_path not in sys.path:
    sys.path.append(module_path)
import morphology_networks as net
import model_training as train
import model_testing as test
import physically_informed_loss_functions as PhysLoss
import network_utils as nuts

torch.manual_seed(28)

In [2]:
test_data_bucket = 'sagemaker-us-east-2-362637960691'
test_data_path = 'py-conjugated/m2py_labels/OPV_labels/test_set/'
model_states_path = 'py-conjugated/model_states/OPV/'

In [3]:
sagemaker_session = sagemaker.Session()
session_bucket = sagemaker_session.default_bucket()
role = sagemaker.get_execution_role()

training_image = get_image_uri(sagemaker_session.boto_region_name, 'image-classification', repo_version="latest")

In [7]:
%autoreload

test_dict, test_label_df = nuts.load_s3_ims(test_data_bucket, test_data_path)

In [8]:
hyperparameters = {
    'epochs': 100,
    'lr': 0.004
}

now = str(int(time.time()))
job_name = f'OPVCNN-train-{now}'

In [9]:
estimator = PyTorch(entry_point = 'train_test_script.py',
                    source_dir = '../py-conjugated/networks/',
                    role = role,
                    framework_version = '1.5.0',
                    train_instance_count = 1,
                    train_instance_type = 'ml.p2.xlarge',
                    hyperparameters = hyperparameters,
                    sagemaker_session = sagemaker_session)


In [11]:
estimator.fit(test_data_set, job_name = job_name)

'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


ValueError: Cannot format input <torch.utils.data.dataloader.DataLoader object at 0x7f6620ed8278>. Expecting one of str, dict, s3_input or FileSystemInput

In [ ]:
deployed_endpoint = estimator.deploy(
    initial_instance_count = 1,
    instance_type = 'ml.t2.medium'
    update_endpoint = True)

In [ ]:
for key, im in im_dict.items():
    print(f'predicting image #: {key}')
    
    im_tensor = torch.from_numpy(im).float()
    im_tensor = im_tensor.view(2, 256, 256)
    im_pred = deployed_endpoint.predict(im_tensor)
    
    print(f'prediction: {im_pred}')
    print('----------------------------')

In [ ]:
deployed_endpoint.delete_endpoint()

In [ ]:
fig, ax = plt.subplots(figsize = (8,6))

epochs = np.arange(1, (num_epochs+1), 1)

plt.plot(epochs, im_train_epoch_losses, c = 'k', label = 'training error')
plt.plot(epochs, im_test_epoch_losses, c = 'r', label = 'testing error')
plt.legend(loc = 'upper right')
plt.show()